In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os
import cv2
import numpy as np
from keras.utils import to_categorical

# Define the directory where the folders of images are located
directory = '/content/drive/My Drive/Dataset_Raw/Lung_Opacity_vs_Normal/'

# Define the desired size for the images
image_size = (28, 28)

# Define the mapping of folder names to numerical labels
class_dict = {'Normal': 0, 'Lung_Opacity': 1}

# Define the number of classes
num_classes = len(class_dict)

# Load the images and preprocess them
data = []
labels = []
for class_name in os.listdir(directory):
    class_directory = os.path.join(directory, class_name)
    if class_name in class_dict:
        class_label = class_dict[class_name]
        for filename in os.listdir(class_directory):
            if filename.endswith('.png'):
                img = cv2.imread(os.path.join(class_directory, filename))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # convert to grayscale
                img = cv2.resize(img, image_size)  # resize the image
                data.append(img)
                labels.append(class_label)

# Convert the data and labels to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Reshape the data to have a single channel (grayscale)
data = np.reshape(data, (*data.shape, 1))

# Convert the labels to one-hot encoding
labels = to_categorical(labels, num_classes)



In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.naive_bayes import GaussianNB

# Define the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the CNN model on the training set
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

# Extract features from the CNN model for the training and testing sets
train_features = model.predict(x_train)
test_features = model.predict(x_test)

nb = GaussianNB()
nb.fit(train_features, np.argmax(y_train, axis=1))

Epoch 1/10
301/301 [==============================] - 11s 32ms/step - loss: 0.6829 - accuracy: 0.7963 - val_loss: 0.3695 - val_accuracy: 0.8478
Epoch 2/10
301/301 [==============================] - 11s 37ms/step - loss: 0.3650 - accuracy: 0.8460 - val_loss: 0.3921 - val_accuracy: 0.8511
Epoch 3/10
301/301 [==============================] - 11s 37ms/step - loss: 0.3385 - accuracy: 0.8613 - val_loss: 0.3579 - val_accuracy: 0.8607
Epoch 4/10
301/301 [==============================] - 9s 29ms/step - loss: 0.3253 - accuracy: 0.8645 - val_loss: 0.3610 - val_accuracy: 0.8611
Epoch 5/10
301/301 [==============================] - 11s 36ms/step - loss: 0.3134 - accuracy: 0.8696 - val_loss: 0.4116 - val_accuracy: 0.8441
Epoch 6/10
301/301 [==============================] - 11s 36ms/step - loss: 0.3026 - accuracy: 0.8789 - val_loss: 0.3324 - val_accuracy: 0.8765
Epoch 7/10
301/301 [==============================] - 9s 31ms/step - loss: 0.2982 - accuracy: 0.8799 - val_loss: 0.3282 - val_accuracy: 0

GaussianNB()

In [ ]:
accuracy = nb.score(test_features, np.argmax(y_test, axis=1))
print('Naive Bayes classifier accuracy:', accuracy)

Naive Bayes classifier accuracy: 0.95
